# Surprise homework

In [1]:
import io
import pandas as pd
from surprise import get_dataset_dir

from collections import defaultdict

### Functions

In [2]:
from surprise import Dataset
from surprise import SVD
from surprise import accuracy

from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')

In [4]:
K = 30
trainset, testset = train_test_split(data, test_size=.25)

### Define algorithms

In [5]:
from surprise import NormalPredictor
from surprise import SVD
from surprise import KNNBasic

algorithms_rsma = {}
algorithms = {
    "n_p": NormalPredictor(),
    "kNN_cos": KNNBasic(k = K, sim_options = {'name': 'cosine'}),
    "kNN_MSD": KNNBasic(k = K),
    "kNN_Pearson": KNNBasic(k = K, sim_options = {'name': 'pearson'}),
    "SVD" : SVD()
}

### Select algorithm

In [6]:
for [name, algo] in algorithms.items():
    cv = cross_validate(algo, data, measures=['RMSE'], verbose=True)
    algorithms_rsma[name] = cv['test_rmse'].mean()

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5281  1.5270  1.5102  1.5263  1.5176  1.5218  0.0069  
Fit time          0.09    0.11    0.12    0.13    0.11    0.11    0.01    
Test time         0.14    0.11    0.15    0.17    0.12    0.14    0.02    
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0114  1.0149  1.0171  1.0259  1.0231  1.0185  0.0053  
Fit time          1.50    1.50    1.51    1.48    1.50 

### Calculate precision@k and recall@k

In [7]:
#Определим самый быстрый алгоритм
fastest_algorithm = min(algorithms_rsma.items(), key=lambda i: i[1])
print(fastest_algorithm)
precision_algorithm = algorithms[fastest_algorithm[0]]

def precision_recall_at_k(predictions, k, threshold):
    "Return precision and recall at k metrics for each user"

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls
precision_algorithm.fit(trainset)
predictions = precision_algorithm.test(testset)

precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)

# Precision and recall averaged over all users
precision_at_k = sum(prec for prec in precisions.values()) / len(precisions)
recall_at_k = sum(rec for rec in recalls.values()) / len(recalls)

print(precision_at_k)
print(recall_at_k)

('SVD', 0.9354326161299209)
0.7589961117002478
0.36829599628734816


### Predict

In [8]:
# обратите внимание на функцию build_anti_testset
k = 5
user_index = '16'

precision_algorithm.fit(trainset)
testset = trainset.build_anti_testset()
predictions = precision_algorithm.test(testset)

user_predictions = list(filter(lambda x: x.uid == user_index, predictions))
unrated_tuple = sorted(user_predictions, key=lambda x: x.est, reverse=True)[0:k]

def get_movie_info(urated_tuple):
    data = pd.read_csv(get_dataset_dir() + '/ml-100k/ml-100k/u.item', sep='|',encoding='ISO-8859-1', header = None) 
    movie_info = {}
    for movie_id in urated_tuple:
        row = data.iloc[int(movie_id)]
        movie_info[movie_id] = (row[1], row[2])
    return movie_info
movie_info = get_movie_info(list(map(lambda x: x[1], unrated_tuple)))

### Запись в файл

In [9]:
file = open("task2.2_Korop.txt", "w")
file.write('User ' + user_index + "\n")
print('User ' + user_index)
for movie in unrated_tuple:
    file.write('{}, {}, {}'.format(movie[1], movie_info[movie[1]], round(movie[3], 3)))
    file.write("\n")
    print(movie[1],movie_info[movie[1]], movie[3])
file.close()

User 16
603 ('It Happened One Night (1934)', '01-Jan-1934') 5
318 ('Everyone Says I Love You (1996)', '06-Dec-1996') 5
408 ('Jack (1996)', '07-Aug-1996') 5
313 ('3 Ninjas: High Noon At Mega Mountain (1998)', '01-Jan-1997') 5
251 ('Lost World: Jurassic Park, The (1997)', '23-May-1997') 5
